<a href="https://colab.research.google.com/github/BimalGhimire38/Extra_Python_Stuffs/blob/main/Case_Study_Mapping_Working_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install geopandas matplotlib contextily -q

In [14]:
import geopandas as gpd
import matplotlib.pyplot as plt
import contextily as cx
import os
from matplotlib.ticker import FuncFormatter, MultipleLocator

# 10 no cell

In [10]:
# Set paths
base_path = '/content/sample_data/case_study_maps/'

# Load layers
districts = gpd.read_file(base_path + 'study_area_districts.gpkg')
municipalities = gpd.read_file(base_path + 'municipalities.gpkg')
selected_muni = gpd.read_file(base_path + 'selected_local_levels.gpkg')
drought_cases = gpd.read_file(base_path + 'drought_case_studies.gpkg')
flood_cases = gpd.read_file(base_path + 'flood_case_studies.gpkg')

# Create output folder
os.makedirs('/content/output_maps', exist_ok=True)

# Function to format deg + min (whole minutes, no decimal)
def deg_to_dm(deg):
    degrees = int(deg)
    minutes = int((deg - degrees) * 60)
    return f"{degrees}°{minutes}'"

# List of cases with extra info
cases = [
    {'id': 'DR-1', 'type': 'drought', 'muni': 'kaudena', 'ward': '06', 'village': 'Sakraut', 'district': 'Sarlahi'},
    {'id': 'DR-2', 'type': 'drought', 'muni': 'kaudena', 'ward': '06', 'village': 'Sakraut', 'district': 'Sarlahi'},
    {'id': 'DR-3', 'type': 'drought', 'muni': 'parsa', 'ward': '06', 'village': 'Muslim Tole', 'district': 'Sarlahi'},
    {'id': 'DR-4', 'type': 'drought', 'muni': 'bramhapuri', 'ward': '05', 'village': 'Nakilwa', 'district': 'Sarlahi'},
    {'id': 'DR-5', 'type': 'drought', 'muni': 'chandrapur', 'ward': '02', 'village': 'Nursery Tole', 'district': 'Rautahat'},
    {'id': 'DR-6', 'type': 'drought', 'muni': 'chandrapur', 'ward': '02', 'village': 'Tempo Park', 'district': 'Rautahat'},
    {'id': 'DR-7', 'type': 'drought', 'muni': 'rajpur', 'ward': '02', 'village': 'Phatuha', 'district': 'Rautahat'},
    {'id': 'DR-8', 'type': 'drought', 'muni': 'rajdevi', 'ward': '08', 'village': 'Laxmipur', 'district': 'Rautahat'},
    {'id': 'DR-9', 'type': 'drought', 'muni': 'durga', 'ward': '05', 'village': 'Badharwa', 'district': 'Rautahat'},
    {'id': 'DR-10', 'type': 'drought', 'muni': 'durga', 'ward': '04', 'village': 'Pacharukhi', 'district': 'Rautahat'},
    {'id': 'DR-11', 'type': 'drought', 'muni': 'paroha', 'ward': '03', 'village': 'Gharari Tole', 'district': 'Rautahat'},
    {'id': 'DR-12', 'type': 'drought', 'muni': 'ishanath', 'ward': '06', 'village': 'Ghiura', 'district': 'Rautahat'},
    {'id': 'FL-1', 'type': 'flood', 'muni': 'kaudena', 'ward': '06', 'village': 'Sakraut', 'district': 'Sarlahi'},
    {'id': 'FL-2', 'type': 'flood', 'muni': 'kaudena', 'ward': '06', 'village': 'Sakraut', 'district': 'Sarlahi'},
    {'id': 'FL-3', 'type': 'flood', 'muni': 'rajpur', 'ward': '02', 'village': 'Phatuha', 'district': 'Rautahat'},
    {'id': 'FL-4', 'type': 'flood', 'muni': 'rajpur', 'ward': '05', 'village': 'Farhadawa', 'district': 'Rautahat'},
    {'id': 'FL-5', 'type': 'flood', 'muni': 'rajdevi', 'ward': '08', 'village': 'Laxmipur', 'district': 'Rautahat'},
    {'id': 'FL-6', 'type': 'flood', 'muni': 'durga', 'ward': '05', 'village': 'Badharwa', 'district': 'Rautahat'},
    {'id': 'FL-7', 'type': 'flood', 'muni': 'durga', 'ward': '04', 'village': 'Pacharukhi', 'district': 'Rautahat'},
    {'id': 'FL-8', 'type': 'flood', 'muni': 'paroha', 'ward': '03', 'village': 'Gharari Tole', 'district': 'Rautahat'},
    {'id': 'FL-9', 'type': 'flood', 'muni': 'gaur', 'ward': '04', 'village': 'Tikulia', 'district': 'Rautahat'},
    {'id': 'FL-10', 'type': 'flood', 'muni': 'madhav', 'ward': '08', 'village': 'Pipara Rajwada', 'district': 'Rautahat'}
]

# District column
district_col = 'DISTRICT'  # Adjust if different

# Loop over each case
for case in cases:
    case_id = case['id']
    case_type = case['type']
    muni_name = case['muni']
    ward = case['ward']
    village = case['village']
    district = case['district']

    # Get point
    if case_type == 'drought':
        point = drought_cases[drought_cases['ID'] == case_id]
    else:
        point = flood_cases[flood_cases['ID'] == case_id]

    # Get active municipality (match on first word lowercase)
    active_muni = selected_muni[selected_muni[muni_col].str.split(' ').str[0].str.lower() == muni_name]

    # Create figure
    fig, ax = plt.subplots(figsize=(7, 7))

    # Plot districts with different fills and thicker boundaries
    rautahat = districts[districts[district_col] == 'RAUTAHAT']
    sarlahi = districts[districts[district_col] == 'SARLAHI']
    rautahat.plot(ax=ax, color='lightyellow', edgecolor='black', linewidth=2)
    sarlahi.plot(ax=ax, color='lightgreen', edgecolor='black', linewidth=2)

    # Plot municipalities
    municipalities.plot(ax=ax, color='white', edgecolor='lightgray', linewidth=0.5)

    # Highlight active municipality
    if not active_muni.empty:
        active_muni.plot(ax=ax, color='lightblue', edgecolor='red', linewidth=2, alpha=0.5)

    # Plot case study as star
    if not point.empty:
        point.plot(ax=ax, color='darkblue', marker='*', markersize=250, edgecolor='white', linewidth=0.75, label=f'{case_id}: {village}, Ward {ward}, {district}')

    # Add basemap without attribution
    cx.add_basemap(ax, crs=point.crs.to_string(), source=cx.providers.OpenStreetMap.Mapnik, attribution=False, zoom=10)

    # Title
    ax.set_title(f'Location Map for {case_id}', fontsize=14, pad=20)

    # Frame with coordinates (multiples of 5 minutes)
    ax.set_xlabel('Longitude (°E)')
    ax.set_ylabel('Latitude (°N)')
    ax.tick_params(axis='both', which='major', labelsize=10)
    ax.grid(True, linestyle='--', alpha=0.5)

    # Set multiple of 5 minutes (5/60 degrees)
    ax.xaxis.set_major_locator(MultipleLocator(5/60))
    ax.yaxis.set_major_locator(MultipleLocator(5/60))

    # Formatter for deg + min (whole minutes)
    ax.xaxis.set_major_formatter(FuncFormatter(lambda x, _: deg_to_dm(x)))
    ax.yaxis.set_major_formatter(FuncFormatter(lambda y, _: deg_to_dm(y)))

    # Legend with smaller star in label (use handles to control size)
    from matplotlib.lines import Line2D
    star_handle = Line2D([], [], color='darkblue', marker='*', linestyle='None', markersize=8, label=f'{case_id}: {village}, Ward {ward}, {district}')
    ax.legend(handles=[star_handle], loc='upper right', fontsize=10, frameon=True, edgecolor='black')

    # Square aspect and tight layout
    ax.set_aspect('equal')
    plt.tight_layout()

    # Save
    plt.savefig(f'/content/output_maps/{case_id}_location_map.png', dpi=300, bbox_inches='tight')
    plt.close()

# List generated maps
print("Generated maps:")
print(os.listdir('/content/output_maps'))

Generated maps:
['DR-9_location_map.png', 'DR-4_location_map.png', 'FL-1_location_map.png', 'DR-3_location_map.png', 'DR-12_location_map.png', 'DR-2_location_map.png', 'FL-8_location_map.png', 'DR-5_location_map.png', 'DR-8_location_map.png', 'FL-4_location_map.png', 'DR-7_location_map.png', 'FL-7_location_map.png', 'DR-11_location_map.png', 'DR-10_location_map.png', 'FL-5_location_map.png', 'FL-2_location_map.png', 'FL-3_location_map.png', 'FL-6_location_map.png', 'DR-1_location_map.png', 'FL-10_location_map.png', 'FL-9_location_map.png', 'DR-6_location_map.png']


# New Revision

In [31]:
# Set paths
base_path = '/content/sample_data/case_study_maps/'

# Load layers
districts = gpd.read_file(base_path + 'study_area_districts.gpkg')
municipalities = gpd.read_file(base_path + 'municipalities.gpkg')
selected_muni = gpd.read_file(base_path + 'selected_local_levels.gpkg')
drought_cases = gpd.read_file(base_path + 'drought_case_studies.gpkg')
flood_cases = gpd.read_file(base_path + 'flood_case_studies.gpkg')

# Create output folder
os.makedirs('/content/output_maps', exist_ok=True)

# Function to format deg + min (whole minutes, no decimal)
def deg_to_dm(deg):
    degrees = int(deg)
    minutes = int((deg - degrees) * 60)
    return f"{degrees}°{minutes}'"

# List of cases with extra info
cases = [
    {'id': 'DR-1', 'type': 'drought', 'muni': 'Kaudena Rural Municipality', 'ward': '06', 'village': 'Sakraut', 'district': 'Sarlahi'},
    {'id': 'DR-2', 'type': 'drought', 'muni': 'Kaudena Rural Municipality', 'ward': '06', 'village': 'Sakraut', 'district': 'Sarlahi'},
    {'id': 'DR-3', 'type': 'drought', 'muni': 'Parsa Rural Municipality', 'ward': '06', 'village': 'Muslim Tole', 'district': 'Sarlahi'},
    {'id': 'DR-4', 'type': 'drought', 'muni': 'Bramhapuri Rural Municipality', 'ward': '05', 'village': 'Nakilwa', 'district': 'Sarlahi'},
    {'id': 'DR-5', 'type': 'drought', 'muni': 'Chandrapur Municipality', 'ward': '02', 'village': 'Nursery Tole', 'district': 'Rautahat'},
    {'id': 'DR-6', 'type': 'drought', 'muni': 'Chandrapur Municipality', 'ward': '02', 'village': 'Tempo Park', 'district': 'Rautahat'},
    {'id': 'DR-7', 'type': 'drought', 'muni': 'Rajpur Municipality', 'ward': '02', 'village': 'Phatuha', 'district': 'Rautahat'},
    {'id': 'DR-8', 'type': 'drought', 'muni': 'Rajdevi Municipality', 'ward': '08', 'village': 'Laxmipur', 'district': 'Rautahat'},
    {'id': 'DR-9', 'type': 'drought', 'muni': 'Durga Bhagwati Rural Municipality', 'ward': '05', 'village': 'Badharwa', 'district': 'Rautahat'},
    {'id': 'DR-10', 'type': 'drought', 'muni': 'Durga Bhagwati Rural Municipality', 'ward': '04', 'village': 'Pacharukhi', 'district': 'Rautahat'},
    {'id': 'DR-11', 'type': 'drought', 'muni': 'Paroha Municipality', 'ward': '03', 'village': 'Gharari Tole', 'district': 'Rautahat'},
    {'id': 'DR-12', 'type': 'drought', 'muni': 'Ishanath Municipality', 'ward': '06', 'village': 'Ghiura', 'district': 'Rautahat'},
    {'id': 'FL-1', 'type': 'flood', 'muni': 'Kaudena Rural Municipality', 'ward': '06', 'village': 'Sakraut', 'district': 'Sarlahi'},
    {'id': 'FL-2', 'type': 'flood', 'muni': 'Kaudena Rural Municipality', 'ward': '06', 'village': 'Sakraut', 'district': 'Sarlahi'},
    {'id': 'FL-3', 'type': 'flood', 'muni': 'Rajpur Municipality', 'ward': '02', 'village': 'Phatuha', 'district': 'Rautahat'},
    {'id': 'FL-4', 'type': 'flood', 'muni': 'Rajpur Municipality', 'ward': '05', 'village': 'Farhadawa', 'district': 'Rautahat'},
    {'id': 'FL-5', 'type': 'flood', 'muni': 'Rajdevi Municipality', 'ward': '08', 'village': 'Laxmipur', 'district': 'Rautahat'},
    {'id': 'FL-6', 'type': 'flood', 'muni': 'Durga Bhagwati Rural Municipality', 'ward': '05', 'village': 'Badharwa', 'district': 'Rautahat'},
    {'id': 'FL-7', 'type': 'flood', 'muni': 'Durga Bhagwati Rural Municipality', 'ward': '04', 'village': 'Pacharukhi', 'district': 'Rautahat'},
    {'id': 'FL-8', 'type': 'flood', 'muni': 'Paroha Municipality', 'ward': '03', 'village': 'Gharari Tole', 'district': 'Rautahat'},
    {'id': 'FL-9', 'type': 'flood', 'muni': 'Gaur Municipality', 'ward': '04', 'village': 'Tikulia', 'district': 'Rautahat'},
    {'id': 'FL-10', 'type': 'flood', 'muni': 'Madhav Narayan Municipality', 'ward': '08', 'village': 'Pipara Rajwada', 'district': 'Rautahat'}
]

# District column
district_col = 'DISTRICT'  # Adjust if different

# Loop over each case
for case in cases:
    case_id = case['id']
    case_type = case['type']
    muni_name = case['muni']
    ward = case['ward']
    village = case['village']
    district = case['district']

    # Get point
    if case_type == 'drought':
        point = drought_cases[drought_cases['ID'] == case_id]
    else:
        point = flood_cases[flood_cases['ID'] == case_id]

    # Get active municipality (match on first word lowercase)
    active_muni = selected_muni[selected_muni['local_level_name'].str.split(' ').str[0].str.lower() == muni_name.split()[0].lower()]

    # Create figure
    fig, ax = plt.subplots(figsize=(7, 7))

    # Plot districts with different fills and thicker boundaries
    rautahat = districts[districts[district_col] == 'RAUTAHAT']
    sarlahi = districts[districts[district_col] == 'SARLAHI']
    rautahat.plot(ax=ax, color='lightyellow', edgecolor='black', linewidth=2)
    sarlahi.plot(ax=ax, color='lightgreen', edgecolor='black', linewidth=2)


    # Plot municipalities
    municipalities.plot(ax=ax, color='white', edgecolor='lightgray', linewidth=0.5)

    # Highlight active municipality
    if not active_muni.empty:
        active_muni.plot(ax=ax, color='lightblue', edgecolor='red', linewidth=1.25, alpha=0.5)

    # Plot case study as star
    if not point.empty:
        point.plot(ax=ax, color='darkblue', marker='*', markersize=150, edgecolor='white', linewidth=0.75, label=f'{case_id}: {village}, Ward {ward}, {district}')

    # Add basemap without attribution
    cx.add_basemap(ax, crs=point.crs.to_string(), source=cx.providers.OpenStreetMap.Mapnik, attribution=False, zoom=10)

    # Plot districts with different fills and thicker boundaries
    # rautahat = districts[districts[district_col] == 'RAUTAHAT']
    # sarlahi = districts[districts[district_col] == 'SARLAHI']
    # rautahat.plot(ax=ax, color='lightyellow', edgecolor='black', alpha=0, linewidth=2)
    # sarlahi.plot(ax=ax, color='lightgreen', edgecolor='black', alpha=0, linewidth=2)

    # Title
    ax.set_title(f'Location Map for {case_id}', fontsize=14, pad=20)

    # Frame with coordinates (multiples of 5 minutes)
    ax.set_xlabel('Longitude (°E)')
    ax.set_ylabel('Latitude (°N)')
    ax.tick_params(axis='both', which='major', labelsize=10)
    ax.grid(True, linestyle='--', alpha=0.5)

    # Set multiple of 5 minutes (5/60 degrees)
    ax.xaxis.set_major_locator(MultipleLocator(5/60))
    ax.yaxis.set_major_locator(MultipleLocator(5/60))

    # Formatter for deg + min (whole minutes)
    ax.xaxis.set_major_formatter(FuncFormatter(lambda x, _: deg_to_dm(x)))
    ax.yaxis.set_major_formatter(FuncFormatter(lambda y, _: deg_to_dm(y)))

    # Legend with smaller star in label (use handles to control size)
    from matplotlib.lines import Line2D
    star_handle = Line2D([], [], color='darkblue', marker='*', linestyle='None', markersize=8, label=f'{case_id}: {muni_name}, Ward {ward}, {district}')
    ax.legend(handles=[star_handle], loc='upper right', fontsize=10, frameon=True, edgecolor='black')

    # Square aspect and tight layout
    ax.set_aspect('equal')
    plt.tight_layout()

    # Save
    plt.savefig(f'/content/output_maps/{case_id}_location_map.png', dpi=300, bbox_inches='tight')
    plt.close()

# List generated maps
print("Generated maps:")
print(os.listdir('/content/output_maps'))

Generated maps:
['DR-9_location_map.png', 'DR-4_location_map.png', 'FL-1_location_map.png', 'DR-3_location_map.png', 'DR-12_location_map.png', 'DR-2_location_map.png', 'FL-8_location_map.png', 'DR-5_location_map.png', 'DR-8_location_map.png', 'FL-4_location_map.png', 'DR-7_location_map.png', 'FL-7_location_map.png', 'DR-11_location_map.png', 'DR-10_location_map.png', 'FL-5_location_map.png', 'FL-2_location_map.png', 'FL-3_location_map.png', 'FL-6_location_map.png', 'DR-1_location_map.png', 'FL-10_location_map.png', 'FL-9_location_map.png', 'DR-6_location_map.png']


In [32]:
!zip -r /content/output_maps.zip /content/output_maps

from google.colab import files
files.download('/content/output_maps.zip')

updating: content/output_maps/ (stored 0%)
updating: content/output_maps/DR-9_location_map.png (deflated 2%)
updating: content/output_maps/DR-4_location_map.png (deflated 2%)
updating: content/output_maps/FL-1_location_map.png (deflated 2%)
updating: content/output_maps/DR-3_location_map.png (deflated 2%)
updating: content/output_maps/DR-12_location_map.png (deflated 2%)
updating: content/output_maps/DR-2_location_map.png (deflated 2%)
updating: content/output_maps/FL-8_location_map.png (deflated 2%)
updating: content/output_maps/DR-5_location_map.png (deflated 2%)
updating: content/output_maps/DR-8_location_map.png (deflated 2%)
updating: content/output_maps/FL-4_location_map.png (deflated 2%)
updating: content/output_maps/DR-7_location_map.png (deflated 2%)
updating: content/output_maps/FL-7_location_map.png (deflated 2%)
updating: content/output_maps/DR-11_location_map.png (deflated 2%)
updating: content/output_maps/DR-10_location_map.png (deflated 2%)
updating: content/output_maps/

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

2082/09/29 11:28 PM,    01/13/2026 .. @ Har Har Mahadev